In [3]:
import pandas as pd
import numpy as np
import time
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [4]:
df = pd.read_parquet("../data/full_dataset_feature_engineering_v1.parquet")

In [5]:
df

,news_count,return_forward,sma_5_20_diff,sma_20_50_diff,sma_50_200_diff,ema_5_20_diff,ema_20_50_diff,ema_50_200_diff,sma_5_20_diff_abs_chg,sma_5_20_diff_pct_chg,...,NER_michael saylor_last24,NER_microstrategy_last3,NER_microstrategy_last24,NER_sec_last3,NER_sec_last24,NER_us_last3,reddit_total_sentiment_ewm_5h,reddit_total_sentiment_ewm_24h,reddit_average_sentiment_ewm_5h,reddit_average_sentiment_ewm_24h
2021-01-01 06:00:00,2.0,-0.007795,219.4000,351.4194,2328.15940,253.706224,434.108627,1931.153143,30.9855,0.164454,...,0,0,0,0,0,0,-0.891000,-0.891000,-0.111375,-0.111375
2021-01-01 07:00:00,0.0,0.003292,253.0935,336.2693,2332.87565,247.173223,435.446110,1934.233818,33.6935,0.153571,...,0,0,0,0,0,0,-0.594000,-0.819720,-0.074250,-0.102465
2021-01-01 08:00:00,2.0,-0.001883,284.9410,309.6284,2338.85115,258.314486,440.377401,1939.227744,31.8475,0.125833,...,1,1,1,0,0,0,-0.396000,-0.754142,-0.049500,-0.094268
2021-01-01 09:00:00,2.0,-0.001648,326.5890,301.5680,2335.72595,243.679390,439.924107,1941.604473,41.6480,0.146164,...,1,1,1,0,0,0,-0.264000,-0.693811,-0.033000,-0.086726
2021-01-01 10:00:00,3.0,0.000118,351.1670,289.6874,2330.35155,215.620781,435.182264,1941.694715,24.5780,0.075257,...,1,1,1,0,0,0,-0.176000,-0.638306,-0.022000,-0.079788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,7.0,-0.006308,339.3000,312.8600,-1716.76500,92.250077,114.783572,-1486.632740,-329.9500,-0.493015,...,1,1,1,1,1,1,0.369066,0.208662,0.026292,0.015598
2024-12-31 20:00:00,6.0,0.002858,-30.2000,309.9000,-1704.44000,-105.563575,70.973167,-1489.015782,-369.5000,-1.089007,...,1,0,1,1,1,1,0.246044,0.191969,0.017528,0.014350
2024-12-31 21:00:00,3.0,-0.004142,-238.4500,313.7300,-1694.03500,-157.957093,47.582523,-1482.887694,-208.2500,6.895695,...,1,0,1,1,1,1,0.164029,0.176611,0.011685,0.013202
2024-12-31 22:00:00,4.0,0.000933,-365.3000,310.7600,-1696.71500,-276.925974,5.335072,-1487.834867,-126.8500,0.531977,...,1,0,1,0,1,1,0.319753,0.212978,0.017354,0.014441
